In [ ]:
import os
import json
import importlib

import glob as gb
import utils as ut
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as pyw
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML, clear_output

# Global

In [ ]:
%matplotlib inline

In [ ]:
importlib.reload(ut)

plt.rcParams['text.usetex'] = True
plt.style.use(['dark_background'])
plt.style.use(['default'])

# Data

In [ ]:
# load data
data = ut.load_data(os.path.join('data', 'v8-ortho', 'forest-23*', '*.zip'))

In [ ]:
# simulation
index = 0
simulation = list(data.keys())[index]

print('simulations:\n', '\n '.join([f'{i}: ' + (f'{x} [x]' if x == simulation else x) for i, x in enumerate(list(data.keys()))]))

In [ ]:
# load images
df = data[simulation]['images']
df = df[df['type'] == 'monochrome']
df = df.reset_index(drop=True)

print('images:', df.shape[0])

In [ ]:
# load parameters
parameters = data[simulation]['parameters']
parameters['images'] = df.shape[0]

print('\nparameters:', json.dumps(parameters, indent=4))

# Plots

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))

# plot stage image
ut.plot_image(ax, data[simulation]['stage'], 'stage')

# plot sample images
ut.plot_images(df['data'], df['name'])

In [ ]:
df_integrate = df[:18]

# integrate images
integrated = ut.integrate_image(df_integrate, parameters, N=30)

# grayscale images
grayscaled = np.array([ut.grayscale_image(x) for x in df_integrate['data']])

In [ ]:
image_vs_integrated = np.hstack([grayscaled, integrated]).reshape(integrated.shape * np.array([2, 1, 1]))
image_vs_integrated_labels = np.dstack([df_integrate['name'], df_integrate['name'] + '-integral']).flatten()

# plot integrated images
ut.plot_images(image_vs_integrated, image_vs_integrated_labels, rows=6, cols=6)

In [ ]:
# integrate ground
ground, alphas = ut.integrate_ground(df, parameters)

# aggregate alphas
df_alpha = ut.aggregate_alphas(alphas, sample=None)

# calculate ground visibility
scanned = np.count_nonzero(ground[:, :, 0])
captured = np.count_nonzero(ground[:, :, 1])
visibility = captured / scanned

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(24, 6))

# plot ground
ut.plot_heatmap(axs[0], ground[:, :, 0], 'scanned pixels (count)')
ut.plot_heatmap(axs[1], ground[:, :, 1], 'visible pixels (count)')
ut.plot_heatmap(axs[2], ut.normalize_image(ground[:, :, 1] > 0), f'visibility ({visibility:.2f})')

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(24, 6))

# plot alpha
for i, column in enumerate(['scanned', 'visible', 'ratio']):
    df_alpha.plot(kind='line', x='alpha', y=column, label=f'{column} (mean)', ax=axs[i])

In [ ]:
# density
density = 1 - visibility
mean = 1 - (ground[:, :, 1] > 0).mean()

print('density:', density)
print('mean:', mean)